In [103]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
import Packages.ClusteringHelper as ch
import numpy as np
from tqdm import tqdm
from collections import Counter
import string
import re
from nltk.corpus import stopwords

In [105]:
text, data = ch.read_aida_yago_conll(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv")
save = False
if save:
    text_file = open('text.txt', 'w')
    text_file.write(text)
    text_file.close()
data

,documents,tokens,indexes,word_indexes,mentions,entities,wikidatas,numeric_codes,alpha_codes
0,0,EU,"(0, 2)",0,,,,,
1,0,rejects,"(3, 10)",1,,,,,
2,0,German,"(11, 17)",2,German,Germany,http://en.wikipedia.org/wiki/Germany,11867,/m/0345h
3,0,call,"(18, 22)",3,,,,,
4,0,to,"(23, 25)",4,,,,,
...,...,...,...,...,...,...,...,...,...
285584,1392,younger,"(1342, 1349)",265,,,,,
285585,1392,brother,"(1350, 1357)",266,,,,,
285586,1392,",","(1358, 1359)",267,,,,,
285587,1392,Bobby,"(1360, 1365)",268,Bobby,Bobby_Charlton,http://en.wikipedia.org/wiki/Bobby_Charlton,4224,/m/01c8x


In [106]:
ents_data = data[data['entities'] != ''].copy()

In [107]:
from pathlib import Path

path_train = Path(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\encodings\\AIDA-YAGO2_train_encodings.jsonl")
path_testa = Path(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\encodings\\AIDA-YAGO2_testa_encodings.jsonl")
path_testb = Path(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\encodings\\AIDA-YAGO2_testb_encodings.jsonl")
raw_encodings_train = open(path_train, 'r').read()
raw_encodings_testa = open(path_testa, 'r').read()
raw_encodings_testb = open(path_testb, 'r').read()

In [108]:
import json

jsonl_parsed_train = [json.loads(x) for x in raw_encodings_train.splitlines()]
jsonl_parsed_testa = [json.loads(x) for x in raw_encodings_testa.splitlines()]
jsonl_parsed_testb = [json.loads(x) for x in raw_encodings_testb.splitlines()]
jsonl_parsed = jsonl_parsed_train + jsonl_parsed_testa + jsonl_parsed_testb

In [109]:
encodings = [x['encoding'] for x in jsonl_parsed]

In [110]:
ents_data['encodings'] = encodings
ents_data_filtered = ch.filter_data(ents_data, 3)

In [111]:
ents_data = ents_data_filtered[ents_data_filtered['entities'] != '']
entities = ents_data['entities'].values
mentions = ents_data['tokens'].values
encodings = ents_data['encodings'].values
unique_entities = np.unique(entities)
gold_standard = Counter(entities)
gold_standard = dict(sorted(gold_standard.items(), key=lambda item: item[1], reverse=True))

In [112]:
leven_cluster = np.loadtxt("../aida-yago2-dataset/db_cluster_levestein_0_3.txt", dtype=np.int32)
lev_cluster_dict = {}
for i, x in enumerate(leven_cluster):
    try:
        lev_cluster_dict[x].append(
            (mentions[i], entities[i], encodings[i]))
    except:
        lev_cluster_dict[x] = [
            (mentions[i], entities[i], encodings[i])]

In [113]:
cluster_all = []
from sklearn.cluster import AgglomerativeClustering
clusterizator = AgglomerativeClustering(n_clusters=None, affinity='cosine', distance_threshold=0.03, linkage="single")

for x in tqdm(lev_cluster_dict):
    encodings = [y[2] for y in lev_cluster_dict[x]]
    if len(encodings) > 1:
        cluster = clusterizator.fit_predict(encodings)
        cluster_all.append(cluster)
    else:
        cluster_all.append(np.zeros(1))

100%|██████████| 2750/2750 [00:02<00:00, 1051.57it/s]


In [114]:
total_encodings = [[y[2] for y in lev_cluster_dict[x]] for x in lev_cluster_dict.keys()]

In [115]:
clustered_entities = [[row[1] for row in lev_cluster_dict[index]] for index in lev_cluster_dict.keys()]


In [116]:
clusters_splitted = []
for n_cluster in range(len(clustered_entities)):
    cluster_unique = dict.fromkeys(set(cluster_all[n_cluster]), [])
    for key in cluster_unique:
        cluster_unique[key] = [clustered_entities[n_cluster][index] for index in
                               range(len(clustered_entities[n_cluster])) if cluster_all[n_cluster][index] == key]
    clusters_splitted.append(cluster_unique)

In [117]:
clusters_splitted = []
clusters_vectors = []
for n_cluster in range(len(clustered_entities)):
    cluster_unique = dict.fromkeys(set(cluster_all[n_cluster]), [])
    cluster_unique_vector = dict.fromkeys(set(cluster_all[n_cluster]), [])
    for key in cluster_unique:
        cluster_unique[key] = [clustered_entities[n_cluster][index] for index in
                               range(len(clustered_entities[n_cluster])) if cluster_all[n_cluster][index] == key]
        cluster_unique_vector[key] = [total_encodings[n_cluster][index] for index in
                               range(len(clustered_entities[n_cluster])) if cluster_all[n_cluster][index] == key]
    clusters_vectors.append(cluster_unique_vector)
    clusters_splitted.append(cluster_unique)

In [118]:
clusters_splitted_vector_dict = [np.mean(x[y],axis=0)  for x in clusters_vectors for y in x]

In [119]:
clusters_splitted_dict = [Counter(x[y]) for x in clusters_splitted for y in x]

In [134]:
clusterizator = AgglomerativeClustering(n_clusters=None, affinity='cosine', distance_threshold=0.015, linkage="single")

clusters_of_cluster = clusterizator.fit_predict(clusters_splitted_vector_dict)
len(clusters_of_cluster)

3002

In [135]:
coc_dict = {}
for index, el in enumerate(clusters_of_cluster):
    try:
        coc_dict[el] = coc_dict[el] + clusters_splitted_dict[index]
    except:
        coc_dict[el] = clusters_splitted_dict[index]
len(coc_dict.keys())

2911

In [136]:
coc_dict

{2835: Counter({'Germany': 354,
          'German_language': 3,
          'Germany_national_football_team': 6,
          'Nazi_Germany': 2}),
 10: Counter({'United_Kingdom': 262, 'England': 1}),
 2344: Counter({'Brussels': 43}),
 1997: Counter({'Brussels': 1}),
 2803: Counter({'European_Commission': 8}),
 2125: Counter({'European_Union': 24}),
 1760: Counter({'European_Commission': 11}),
 18: Counter({'France': 299, 'France_national_football_team': 10}),
 177: Counter({'Bovine_spongiform_encephalopathy': 6}),
 1988: Counter({'Budapest_Stock_Exchange': 1}),
 1876: Counter({'Budapest_Stock_Exchange': 1}),
 21: Counter({'Spain': 173, 'Spain_Fed_Cup_team': 4}),
 169: Counter({'Europe': 108,
          'UEFA_Champions_League': 1,
          'European_Union': 8,
          'UEFA_European_Football_Championship': 2}),
 1742: Counter({'Bonn': 42}),
 1631: Counter({'Jimi_Hendrix': 4}),
 2537: Counter({'London': 233,
          'London_Wasps': 2,
          'Harlequins_Rugby_League': 3,
          'Lon

In [137]:
import Packages.ClusteringHelper as ch
golden_standard_dict = ch.get_gold_standard_dict(ents_data)

### B-CUBED

In [138]:
from tqdm import tqdm
# B-cubed - precision
bcubed_precision_num = 0
for gold_key in tqdm(golden_standard_dict.keys()):
    for lev_key in coc_dict.keys():
        try:
            bcubed_precision_num = bcubed_precision_num + (pow(coc_dict[lev_key][gold_key], 2) /
                                       sum(coc_dict[lev_key].values()))
        except:
            pass
bcubed_precision = bcubed_precision_num/sum([sum(x.values()) for x in coc_dict.values()])
bcubed_precision

100%|██████████| 2089/2089 [00:05<00:00, 366.94it/s]


0.8987571069326864

In [139]:
# B-cubed - recall
bcubed_recall_num = 0
for gold_key in tqdm(golden_standard_dict.keys()):
    for lev_key in coc_dict.keys():
        try:
            bcubed_recall_num = bcubed_recall_num + (pow(coc_dict[lev_key][gold_key], 2) /
                                       golden_standard_dict[gold_key])
        except:
            pass
bcubed_recall = bcubed_recall_num/ents_data.shape[0]
bcubed_recall

100%|██████████| 2089/2089 [00:04<00:00, 466.75it/s]


0.8462756606472945

In [140]:
bcubed_f1 = (2 * (bcubed_recall * bcubed_precision)) / (
        bcubed_precision + bcubed_recall)
bcubed_f1

0.8717272002699508